<a href="https://colab.research.google.com/github/Matheusb009/supermarket-sales-eda/blob/main/CommerceIQ_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **"Exploratory Data Analysis of Supermarket Sales"**

## This report presents a comprehensive exploratory data analysis (EDA) of supermarket sales records.
## The primary objective is to uncover sales trends, customer behavior patterns, and product performance across various cities and store branches.

##By examining key factors including revenue, product lines, payment methods, and customer demographics, we aim to generate actionable insights that support strategic business decisions and improve overall performance.

## **Project Overview and Global Configuration**

In [ ]:
# -*- coding: utf-8 -*-
"""
Project: Supermarket Sales - Exploratory Data Analysis
Author: Matheus Bittencourt
Description: This notebook performs a structured EDA to identify sales trends,
             customer behavior patterns, and product performance metrics.
"""

# Global configuration
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.float_format", lambda x: f"{x:,.2f}")
sns.set_theme(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)

## **Version Control and Reproducibility Notes**

In [ ]:
# Version control note (manual)
__version__ = "1.0"
__last_updated__ = "2025-08-13"
__data_source__ = "GitHub RAW"

# For reproducibility
SEED = 42
np.random.seed(SEED)

print(f"Project version: {__version__} | Last updated: {__last_updated__}")

Project version: 1.0 | Last updated: 2025-08-13


# **Data Intake (from GitHub RAW)**

In [2]:


import pandas as pd

DATA_URL = "https://raw.githubusercontent.com/Matheusb009/supermarket-sales-eda/refs/heads/main/supermarket_sales_data.xlsx.csv"

def load_dataset(url: str) -> pd.DataFrame:
    """
    Load dataset from GitHub RAW.
    Tries Excel (openpyxl) first; falls back to CSV with automatic delimiter inference.
    """
    try:
        df = pd.read_excel(url, engine="openpyxl")
        fmt = "Excel"
    except Exception as e_xlsx:
        try:
            df = pd.read_csv(url, engine="python", sep=None, encoding="utf-8")
            fmt = "CSV"
        except Exception as e_csv:
            raise RuntimeError(
                f"Failed to load data.\nExcel error: {type(e_xlsx).__name__}: {e_xlsx}\n"
                f"CSV error: {type(e_csv).__name__}: {e_csv}"
            )
    if df is None or df.empty:
        raise ValueError("Loaded dataframe is empty. Check the DATA_URL or file contents.")
    print(f"[INFO] Loaded as {fmt} | Shape: {df.shape[0]} rows × {df.shape[1]} cols")
    return df

df = load_dataset(DATA_URL)
display(df.head())


[INFO] Loaded as CSV | Shape: 1000 rows × 17 cols


,﻿Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Sales,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,Alex,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,1:08:00 PM,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,Giza,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29:00 AM,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,Alex,Yangon,Normal,Female,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,1:23:00 PM,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,Alex,Yangon,Member,Female,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,8:33:00 PM,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,Alex,Yangon,Member,Female,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37:00 AM,Ewallet,604.17,4.761905,30.2085,5.3


## **Data Validation (Business-Ready Checks)**

In [ ]:
def assert_not_empty(df, name="dataset"):
    if df is None or df.empty:
        raise ValueError(f"[ERROR] {name} is empty after loading.")
    return True

def quick_profile(df):
    prof = {
        "rows": len(df),
        "cols": df.shape[1],
        "missing_rows_pct": float((df.isna().any(axis=1)).mean() * 100),
        "duplicate_rows": int(df.duplicated().sum()),
    }
    return prof

assert_not_empty(df, "raw dataset")
print("[INFO] Quick profile:", quick_profile(df))

[INFO] Quick profile: {'rows': 1000, 'cols': 17, 'missing_rows_pct': 0.0, 'duplicate_rows': 0}


## **Business-Friendly Standardization & Typing**

In [ ]:
# column names
df.columns = (
    df.columns.astype(str)
              .str.strip()
              .str.lower()
              .str.replace(" ", "_")
              .str.replace("%", "pct")
)

# harmonize common fields
if "total" not in df.columns and "sales" in df.columns:
    df = df.rename(columns={"sales": "total"})

# parse date / time
if "date" in df.columns:
    df["date"] = pd.to_datetime(df["date"], errors="coerce", dayfirst=True)

if "time" in df.columns:
    _t = pd.to_datetime(df["time"], format="%H:%M", errors="coerce")
    df["hour"] = _t.dt.hour

# numeric typing
for c in ["unit_price","quantity","tax_5pct","total","cogs",
          "gross_margin_percentage","gross_income","rating"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# categorical tidy
for c in ["branch","city","customer_type","gender","product_line","payment"]:
    if c in df.columns:
        df[c] = df[c].astype(str).str.strip().str.title()

# basic integrity
if "total" in df.columns:
    df = df[df["total"].notna()]

## **Feature Engineering (Time & Business Metrics)**

In [ ]:
# Time-based features
if "date" in df.columns:
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df["year"] = df["date"].dt.year
    df["month"] = df["date"].dt.month
    df["day"] = df["date"].dt.day
    df["weekday"] = df["date"].dt.day_name()
    df["is_weekend"] = df["weekday"].isin(["Saturday", "Sunday"]).astype(int)
    df["year_month"] = df["date"].dt.to_period("M").astype(str)

# Hour from 'time' if available
if "time" in df.columns and "hour" not in df.columns:
    _t = pd.to_datetime(df["time"], format="%H:%M", errors="coerce")
    df["hour"] = _t.dt.hour

# Revenue helpers
if {"total", "tax_5pct"}.issubset(df.columns):
    df["net_revenue"] = df["total"] - df["tax_5pct"]
elif "total" in df.columns and "net_revenue" not in df.columns:
    df["net_revenue"] = df["total"]

# Unit economics
if {"total", "quantity"}.issubset(df.columns):
    df["avg_price_per_item"] = df["total"] / df["quantity"].replace(0, pd.NA)

# Basic sanity print
cols_preview = [c for c in ["year","month","weekday","is_weekend","hour",
                            "net_revenue","avg_price_per_item","year_month"] if c in df.columns]
print("[INFO] Feature engineering added ->", cols_preview)
display(df[cols_preview].head(3) if cols_preview else df.head(3))

[INFO] Feature engineering added -> []


,﻿Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Sales,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,Alex,Yangon,Member,Female,Health and beauty,74.69,7,26.14,548.97,1/5/2019,1:08:00 PM,Ewallet,522.83,4.76,26.14,9.10
1,226-31-3081,Giza,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.82,80.22,3/8/2019,10:29:00 AM,Cash,76.40,4.76,3.82,9.60
2,631-41-3108,Alex,Yangon,Normal,Female,Home and lifestyle,46.33,7,16.22,340.53,3/3/2019,1:23:00 PM,Credit card,324.31,4.76,16.22,7.40


## **Business Data Quality Report**

In [ ]:
def business_quality_report(df):
    checks = {}

    # Key fields missing
    for col in ["total", "quantity", "product_line", "date"]:
        if col in df.columns:
            checks[f"missing_{col}"] = int(df[col].isna().sum())

    # Zero/negative sales
    if "total" in df.columns:
        checks["zero_or_negative_total"] = int((df["total"] <= 0).sum())

    # Date span (ensure datetime first)
    if "date" in df.columns:
        if not pd.api.types.is_datetime64_any_dtype(df["date"]):
            df["date"] = pd.to_datetime(df["date"], errors="coerce")
        m, M = df["date"].min(), df["date"].max()
        checks["min_date"] = str(m.date()) if pd.notna(m) else None
        checks["max_date"] = str(M.date()) if pd.notna(M) else None

    return pd.Series(checks, name="Business Data Quality")

display(business_quality_report(df))

,Business Data Quality


## **Executive KPI Layer**

In [ ]:
kpi = {}
if "total" in df.columns:
    kpi["Total Revenue"]  = float(df["total"].sum())
    kpi["Average Ticket"] = float(df["total"].mean())

if {"total","quantity"}.issubset(df.columns):
    units = df["quantity"].sum()
    kpi["Units Sold"] = float(units)
    if units > 0:
        kpi["Revenue per Unit"] = float(df["total"].sum() / units)

if {"gross_income","total"}.issubset(df.columns) and df["total"].sum() != 0:
    kpi["Gross Margin %"] = round(df["gross_income"].sum() / df["total"].sum() * 100, 2)

kpi_series = pd.Series(kpi, name="Executive KPIs")
display(kpi_series)

,Executive KPIs


## **Dashboard Backbone**

In [4]:
aggs = {}

# Monthly revenue trend
if {"year_month", "total"}.issubset(df.columns):
    aggs["monthly_revenue"] = (
        df.groupby("year_month", as_index=False)["total"]
          .sum()
          .sort_values("year_month")
    )

# Revenue by product line
if {"product_line", "total"}.issubset(df.columns):
    aggs["revenue_by_product"] = (
        df.groupby("product_line", as_index=False)["total"]
          .sum()
          .sort_values("total", ascending=False)
    )

# Revenue by payment method
if {"payment", "total"}.issubset(df.columns):
    aggs["revenue_by_payment"] = (
        df.groupby("payment", as_index=False)["total"]
          .sum()
          .sort_values("total", ascending=False)
    )

# Revenue by city
if {"city", "total"}.issubset(df.columns):
    aggs["revenue_by_city"] = (
        df.groupby("city", as_index=False)["total"]
          .sum()
          .sort_values("total", ascending=False)
    )

# Preview results
for name, a in aggs.items():
    print(f"[INFO] {name}: {a.shape[0]} rows")
    display(a.head(5))

